In [0]:
#!/usr/bin/python3

import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt
import glob
import time

testInputFile = "./testInput.csv"
testTargetFile = "./testTarget.csv"

trainInputFiles = sorted(glob.glob("./trainInput*.csv"))
trainTargetFiles = sorted(glob.glob("./trainTarget*.csv"))

units = range(1, 11)
averageArray = []
timeArray = []

def sort(input):
	return input[0]

def cross():
    unitsMax = []
    for u in units:
        start = time.time()
        averageArrayNeural = []
        for j in range(10):  # 10-fold validation
            testInput = np.loadtxt(trainInputFiles[j], delimiter=",")
            testTarget = np.loadtxt(trainTargetFiles[j], delimiter=",")
            trainInput = np.array([])
            trainTarget = np.array([])
            for l in range(0, j):
                intermediate = np.loadtxt(trainInputFiles[l], delimiter=",")
                trainInput = np.append(trainInput, intermediate)
                intermediate2 = np.loadtxt(trainTargetFiles[l], delimiter=",")
                trainTarget = np.append(trainTarget, intermediate2)
            for l in range(j + 1, 10):
                intermediate = np.loadtxt(trainInputFiles[l], delimiter=",")
                trainInput = np.append(trainInput, intermediate)
                intermediate2 = np.loadtxt(trainTargetFiles[l], delimiter=",")
                trainTarget = np.append(trainTarget, intermediate2)
            trainInput = trainInput.reshape(len(trainTarget), 2)
            testInput = testInput.reshape(len(testTarget), 2)
            neuralLoss = neural(trainInput, trainTarget, testInput, testTarget, u)
            averageArrayNeural.append(neuralLoss)
        averageArray.append(np.mean(averageArrayNeural))
        unitsMax.append([np.mean(averageArrayNeural), u])
        end = time.time()
        timeArray.append(end - start)
    bestU = min(unitsMax, key = sort)[1]
    print("Best number of units:")
    print(bestU)
    print("The loss of the best number of units on the test set:")
    print(bestNeural(bestU))

def neural(trainInput, trainTarget, testInput, testTarget, u):
    model = keras.Sequential()
    model.add(keras.layers.Flatten())
    model.add(keras.layers.Dense(units=u, activation = 'sigmoid'))
    model.add(keras.layers.Dense(units=1, activation = 'linear'))
    # print(len(trainInput))
    # print(len(trainTarget))
    # print(u)
    model.compile(optimizer = 'adam',loss = tf.compat.v1.losses.mean_squared_error)
    model.fit(trainInput, trainTarget, verbose=0)
    loss = model.evaluate(testInput, testTarget)

    return loss

def bestNeural(bestU):
    testInput = np.loadtxt(testInputFile, delimiter=",")
    testTarget = np.loadtxt(testTargetFile, delimiter=",")
    trainInput = np.array([])
    trainTarget = np.array([])
    for inputFile in trainInputFiles:
        intermediate = np.loadtxt(inputFile, delimiter=",")
        trainInput = np.append(trainInput, intermediate)
    for targetFile in trainTargetFiles:
        intermediate = np.loadtxt(targetFile, delimiter=",")
        trainTarget = np.append(trainTarget, intermediate)
    trainInput = trainInput.reshape(len(trainTarget), 2)
    testInput = testInput.reshape(len(testTarget), 2)
    return neural(trainInput, trainTarget, testInput, testTarget, bestU)

cross()
plt.plot(units, averageArray)
plt.savefig("q2dError.png")
plt.plot(units, timeArray)
plt.savefig("q2dTime.png")